### Part 0: Importing Packages

In [1]:
import requests # getting info from API
import json # type of data we're working with from requests
import pandas as pd # sorting the data values

### Part 1: Working with Requests Library

https://developer.riotgames.com/docs/lol#data-dragon_champions    

Within Riot's API for matches, the champions only have Ids and no names associated with them. \
Before we investigate the matches separately, we need to have a dataFrame of all of the champions associated with their id. \
Here, we will be introducing requests and how to use its functions.

In [2]:
resp_chmp = requests.get('http://ddragon.leagueoflegends.com/cdn/10.9.1/data/en_US/champion.json')
r3 = resp_chmp.json()
r3 = r3['data']
chmp_name = []
for i in r3:
    chmp_name.append(i)
x = {}
for i in range(len(r3)):
    name = chmp_name[i]
    x[i]= int(r3[name]['key'])
chmp = pd.DataFrame(chmp_name, columns=['Name'])
chmp = chmp.rename(index=x).sort_index()

### Part 2: Extracting Data from API

https://developer.riotgames.com/apis#summoner-v4

**Task**: Retrieve the match numbers, champions, and lane assignments of the account supplied. Then, we'll extract it onto an Excel file

In [23]:
# Returning the accountId of a user using the summonerName and API Key supplied.
# Inputs:
#   summonerName: Name within League of Legends, case and character sensitive
#   APIKey: API Key from Riot Games 
def accId(summonerName, APIKey):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKey
    response = requests.get(URL)
    r1 = response.json()
    return r1['accountId']

In [25]:
# Find the accountId to get all of the matches from the summoner.
summonerName = (str)(input('Type your Summoner Name here and DO NOT INCLUDE ANY SPACES: '))
APIKey = (str)(input('Copy and paste your API Key here: '))
summ_accId = accId(summonerName, APIKey)
URL = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + summ_accId + "?api_key=" + APIKey
response = requests.get(URL)
r1 = response.json()
r1 = r1['matches']

Type your Summoner Name here and DO NOT INCLUDE ANY SPACES: Phume
Copy and paste your API Key here: RGAPI-6e107c05-f3cf-4004-b645-b9b9c2d7cfd7


In [29]:
r1

[{'platformId': 'NA1',
  'gameId': 3350475946,
  'champion': 40,
  'queue': 420,
  'season': 13,
  'timestamp': 1585539787820,
  'role': 'DUO_SUPPORT',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3350328006,
  'champion': 89,
  'queue': 420,
  'season': 13,
  'timestamp': 1585537864212,
  'role': 'DUO_SUPPORT',
  'lane': 'MID'},
 {'platformId': 'NA1',
  'gameId': 3347884882,
  'champion': 412,
  'queue': 420,
  'season': 13,
  'timestamp': 1585430581043,
  'role': 'DUO_SUPPORT',
  'lane': 'BOTTOM'},
 {'platformId': 'NA1',
  'gameId': 3347780257,
  'champion': 26,
  'queue': 1020,
  'season': 13,
  'timestamp': 1585426195880,
  'role': 'NONE',
  'lane': 'JUNGLE'},
 {'platformId': 'NA1',
  'gameId': 3346359552,
  'champion': 64,
  'queue': 430,
  'season': 13,
  'timestamp': 1585363164184,
  'role': 'SOLO',
  'lane': 'MID'},
 {'platformId': 'NA1',
  'gameId': 3346102975,
  'champion': 44,
  'queue': 420,
  'season': 13,
  'timestamp': 1585354719610,
  'role': 'DUO_SUPPORT',
 

In [30]:
matches = []
champion = []
lane = []
for i in range(len(r1)):
    matches.append(r1[i]['gameId'])
    champ_id = r1[i]['champion']
    champion.append(chmp.loc[champ_id, 'Name'])
    lane.append(r1[i]['lane'])
vals = {'matchId': matches, 'champion': champion, 'lane': lane}
df = pd.DataFrame(vals)

In [32]:
file_path=r'matches.xlsx'
stats=pd.ExcelWriter(file_path)
df.to_excel(stats)
stats.save()
stats.close()

### Intro to working with Game Data

There are many other pieces of data you can look at. Next week we'll be looking at game data and combining everything we have learned to work with it.

In [33]:
gameId = "3225011906"
URL_mch = "https://na1.api.riotgames.com/lol/match/v4/matches/" + gameId + "?api_key=" + APIKey
response = requests.get(URL_mch)
r1 = response.json()
r1 = r1['participants']
r1

[{'participantId': 1,
  'teamId': 100,
  'championId': 83,
  'spell1Id': 12,
  'spell2Id': 4,
  'stats': {'participantId': 1,
   'win': True,
   'item0': 2033,
   'item1': 3111,
   'item2': 3078,
   'item3': 3065,
   'item4': 3053,
   'item5': 1057,
   'item6': 3340,
   'kills': 8,
   'deaths': 1,
   'assists': 6,
   'largestKillingSpree': 6,
   'largestMultiKill': 3,
   'killingSprees': 2,
   'longestTimeSpentLiving': 1253,
   'doubleKills': 2,
   'tripleKills': 1,
   'quadraKills': 0,
   'pentaKills': 0,
   'unrealKills': 0,
   'totalDamageDealt': 216093,
   'magicDamageDealt': 42576,
   'physicalDamageDealt': 164016,
   'trueDamageDealt': 9501,
   'largestCriticalStrike': 0,
   'totalDamageDealtToChampions': 16920,
   'magicDamageDealtToChampions': 5044,
   'physicalDamageDealtToChampions': 11666,
   'trueDamageDealtToChampions': 209,
   'totalHeal': 7748,
   'totalUnitsHealed': 1,
   'damageSelfMitigated': 19181,
   'damageDealtToObjectives': 31913,
   'damageDealtToTurrets': 19316